In [ ]:

%pylab inline

import math
import numpy as np
from scipy.sparse.linalg import inv
#from numpy.linalg import inv
import scipy.sparse as sps
import scipy.sparse.linalg
from scipy import integrate
import sys
import matplotlib.pyplot as plt
sys.path.append('../../src/')
from pylab import *

import parameters as pam
import lattice as lat
import variational_space as vs
import hamiltonian as ham
import basis_change as basis
import get_state as getstate
import utility as util
import plotfig as fig
import ground_state as gs
import lanczos
import time
from tqdm.notebook import tqdm
start_time = time.time()
M_PI = math.pi
                  
#####################################
def compute_Aw_main(A,epx,epy,tpd,tpddiff,tpp,pds,pdsdiff,pdp,pdpdiff,pps,ppp,Upp,Udif,Vpp,Upd,Updf,\
                    d_double,pUppx_double,pUppy_double,pUpp_double,pVpp_double,pUpdx_double,pUpdy_double,U, S_val, Sz_val, AorB_sym): 
    diff = round(abs(epx-epy)*0.5,3)
    if Norb==7:
        fname = 'epx'+str(epx)+'epy'+str(epy)+'_tpd'+str(tpd)+'_tpddiff'+str(tpddiff)+'_tpp'+str(tpp) \
                  +'_A'+str(A)+'_B'+str(pam.B)+'_C'+str(pam.C)+'_Upp'+str(Upp)+'_Udif'+str(Udif)+'_Vpp'\
                  +str(Vpp)+'_Upd'+str(Upd)+'_Updf'+str(Updf) \
                  +'_Mc'+str(Mc)+'_Norb'+str(Norb)+'_eta'+str(eta)
        flowpeak = 'Norb'+str(Norb)+'_tpd'+str(tpd)+'_tpddiff'+str(tpddiff)+'_tpp'+str(tpp)+'_diff'\
                  +str(diff)+'_Upp'+str(Upp)+'_Udif'+str(Udif)\
                  +'_Vpp'+str(Vpp)+'_Upd'+str(Upd)+'_Updf'+str(Updf)+'_Mc'+str(Mc)+'_eta'+str(eta)
    elif Norb==9 or Norb==10 or Norb==11:
        fname = 'epx'+str(epx)+'epy'+str(epy)+'_pds'+str(pds)+'_pdsdiff'+str(pdsdiff)+'_pdp'+str(pdp)\
                  +'_pdpdiff'+str(pdpdiff)+'_pps'+str(pps)+'_ppp'+str(ppp) \
                  +'_A'+str(B)+'_B'+str(pam.B)+'_C'+str(pam.C)+'_Upp'+str(Upp)+'_Udif'+str(Udif)\
                  +'_Vpp'+str(Vpp)+'_Upd'+str(Upd)+'_Updf'+str(Updf)\
                  +'_Mc'+str(Mc)+'_Norb'+str(Norb)+'_eta'+str(eta)
        flowpeak = 'Norb'+str(Norb)+'_pds'+str(pds)+'_pdsdiff'+str(pdsdiff)+'_pdp'+str(pdp)+'_pdpdiff'+str(pdpdiff)\
                  +'_pps'+str(pps)+'_ppp'+str(ppp)+'_diff'+str(diff)+'_Upp'+str(Upp)+'_Udif'+str(Udif)\
                  +'_Vpp'+str(Vpp)+'_Upd'+str(Upd)+'_Updf'+str(Updf)+'_Mc'+str(Mc)+'_eta'+str(eta)
                
    w_vals = np.arange(pam.wmin, pam.wmax, pam.eta/4.0)
   
    Aw = np.zeros(len(w_vals))
    Aw_dd_total = np.zeros(len(w_vals))
    Aw_d8_total = np.zeros(len(w_vals))

    # set up H0
    if Norb==7:
        tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac, tpp_nn_hop_fac \
                                   = ham.set_tpd_tpp(Norb,tpd,tpddiff,tpp,0,0,0,0)
    elif Norb==9 or Norb==11:
        tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac, tpp_nn_hop_fac \
                                   = ham.set_tpd_tpp(Norb,0,0,0,pds,pdsdiff,pdp,pdpdiff,pps,ppp)
            
    T_pd   = ham.create_tpd_nn_matrix(VS,tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac)
    T_pp   = ham.create_tpp_nn_matrix(VS,tpp_nn_hop_fac)  
    Esite  = ham.create_edep_diag_matrix(VS,A,epx,epy)
    
    H0 = T_pd + T_pp + Esite  
    
    '''
    Below probably not necessary to do the rotation by multiplying U and U_d
    the basis_change.py is only for label the state as singlet or triplet
    and assign the interaction matrix
    '''
    if pam.if_H0_rotate_byU==1:
        H0_new = U_d.dot(H0.dot(U))
    
    clf()

    if Norb==7 or Norb==9 or Norb==10 or Norb==11:     
        Hint = ham.create_interaction_matrix_ALL_syms(VS,d_double,pUppx_double,pUppy_double,pUpp_double,pVpp_double,pUpdx_double,\
                                                      pUpdy_double,S_val, Sz_val,AorB_sym,A,Upp,Udif,Vpp,Upd,Updf)
        if pam.if_H0_rotate_byU==1:
            H = H0_new + Hint 
        else:
            H = H0 + Hint 
        H.tocsr()

        ####################################################################################
        # compute GS only for turning on full interactions
        if pam.if_get_ground_state==1:
            vals, vecs, wgt_d8, wgt_d9L, wgt_d10L2 = gs.get_ground_state(H, VS, S_val, Sz_val)
            if Norb==7:
#                 util.write_GS('Egs_'+flowpeak+'.txt',A,epx,epy,tpd,vals[0])
                util.write_GS_components('GS_weights_'+flowpeak+'.txt',A,epx,epy,tpd,wgt_d8, wgt_d9L, wgt_d10L2)
            elif Norb==9 or Norb==10 or Norb==11:
#                 util.write_GS2('Egs_'+flowpeak+'.txt',A,epx,epy,pds,pdp,vals[0])
                util.write_GS_components2('GS_weights_'+flowpeak+'.txt',A,epx,epy,pds,pdp,wgt_d8, wgt_d9L, wgt_d10L2)
            
        #########################################################################
        '''
        Compute A(w) for various states
        '''
        # compute d8
#         fig.compute_Aw_d8_sym(H, VS, d_double, S_val, Sz_val, AorB_sym, A, w_vals, "Aw_d8_sym_", fname)
        
        # compute d9L
#         b1L_state_indices, a1L_state_indices, b1L_state_labels, a1L_state_labels \
#                 = getstate.get_d9L_state_indices(VS, S_val, Sz_val)
#         fig.compute_Aw1(H, VS, w_vals, b1L_state_indices, b1L_state_labels, "Aw_b1L_", fname)
#         fig.compute_Aw1(H, VS, w_vals, a1L_state_indices, a1L_state_labels, "Aw_a1L_", fname)
        
#         # compute d10L2
#         d10L2_state_indices, d10L2_state_labels = getstate.get_d10L2_state_indices(VS, S_val, Sz_val)
#         fig.compute_Aw1(H, VS, w_vals, d10L2_state_indices, d10L2_state_labels, "Aw_d10L2_", fname)
        
##########################################################################
if __name__ == '__main__': 
    Mc  = pam.Mc
    print ('Mc=',Mc)

    Norb = pam.Norb
    eta  = pam.eta
    ed   = pam.ed

    As = pam.As
    B  = pam.B
    C  = pam.C
    
    # set up VS
    VS = vs.VariationalSpace(Mc)
    basis.count_VS(VS)
    
    d_double,pUppx_double,pUppy_double,pUpp_double, pVpp_double, pUpdx_double,pUpdy_double = ham.get_double_occu_list(VS)
    
    # change the basis for d_double states to be singlet/triplet
    if pam.basis_change_type=='all_states':
        U, S_val, Sz_val, AorB_sym = basis.create_singlet_triplet_basis_change_matrix(VS,d_double)
       
        if pam.if_print_VS_after_basis_change==1:
            basis.print_VS_after_basis_change(VS,S_val,Sz_val)
    elif pam.basis_change_type=='d_double':
        U, S_val, Sz_val, AorB_sym = basis.create_singlet_triplet_basis_change_matrix_d_double(VS,d_double)

    U_d = (U.conjugate()).transpose()
    # check if U if unitary
    #checkU_unitary(U,U_d)
    
    if Norb==7:
        
        for tpd in tqdm(pam.tpds):
            for tpddiff in pam.tpddiffs:
                tpddiff = round(tpddiff,3)
                for ep1 in pam.ep_avgs: 
                    for ep2 in pam.ep_difs: 
                        epx = ep1+ep2
                        epy = ep1-ep2
                        for A in pam.As:
                            util.get_atomic_d8_energy(A,B,C)
                            for tpp in pam.tpps:
                                for Upp in pam.Upps:
                                    Upp = round(Upp,3)
                                    for Udif in pam.Udifs:
                                        Udif = round(Udif,3)
                                        for Vpp in pam.Vpps:
                                            Vpp = round(Vpp,3)
                                            for Upd in pam.Upds:
                                                Upd = round(Upd,3)
                                                for Updf in pam.Updfs:
                                                    Updf = round(Updf,3) 
                                                    print ('===================================================')
                                                    print ('A=',A, 'epx=', epx, 'epy=', epy, ' tpd=',tpd,' tpddiff=',tpddiff,' tpp=',tpp, \
                                                           ' Upp=',Upp,'Udif=',Udif,'Vpp=',Vpp,'Upd=',Upd,'Updf=',Updf)
                                                    compute_Aw_main(A,epx,epy,tpd,tpddiff,tpp,0,0,0,0,0,0,Upp,Udif,Vpp,Upd,Updf,\
                                                                    d_double,pUppx_double,pUppy_double,pUpp_double,\
                                                                    pVpp_double,pUpdx_double,\
                                                                    pUpdy_double,U, S_val, Sz_val, AorB_sym)

                                
    elif Norb==9 or Norb==10 or Norb==11:
        total_iterations =len(pam.pdps)*len(pam.pdsdiffs)*len(pam.ep_avgs)*len(pam.ep_difs)*len(pam.As)*\
        len(pam.Upps)*len(pam.Udifs)*len(pam.Vpps)*len(pam.Upds)*len(pam.Updfs)
        pps = pam.pps
        ppp = pam.ppp
        with tqdm(total=total_iterations, desc="Total Progress") as pbar:
            for ii in range(0,len(pam.pdps)):
                pds = pam.pdss[ii]
                pds = round(pds,5)
                pdp = pam.pdps[ii]
                for pdsdiff in pam.pdsdiffs:
                    pdpdiff = round(pdsdiff*np.sqrt(3)/4,5)
                    pdsdiff = round(pdsdiff,3)
                    for ep1 in pam.ep_avgs: 
                        for ep2 in pam.ep_difs: 
                            epx = ep1+ep2
                            epy = ep1-ep2
                            for A in pam.As:
                                util.get_atomic_d8_energy(A,B,C)
                                for Upp in pam.Upps:
                                    Upp = round(Upp,3)
                                    for Udif in pam.Udifs:
                                        Udif = round(Udif,3)
                                        for Vpp in pam.Vpps:
                                            Vpp = round(Vpp,3)
                                            for Upd in pam.Upds:
                                                Upd = round(Upd,3)
                                                for Updf in pam.Updfs:
                                                    Updf = round(Updf,3)
                                                    print ('===================================================')
                                                    print ('A=',A, 'epx=', epx, 'epy=', epy,' pds=',pds,' pdp=',pdp,\
                                                           ' pps=',pps,' ppp=',ppp, ' Upp=',Upp, ' Udif=',Udif,' Vpp=',Vpp ,' Upd=',Upd,)
                                                    compute_Aw_main(A,epx,epy,0,0,0,pds,pdsdiff,pdp,pdpdiff,pps,ppp,Upp,Udif,Vpp,Upd,Updf,\
                                                                    d_double,pUppx_double,pUppy_double,pUpp_double,\
                                                                    pVpp_double,pUpdx_double,\
                                                                    pUpdy_double,\
                                                                    U, S_val, Sz_val, AorB_sym)  
                                                    pbar.update(1)


    print("--- %s seconds ---" % (time.time() - start_time))

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
Ni_orbs =  ['d3z2r2', 'dx2y2', 'dxy', 'dxz', 'dyz']
O1_orbs =  ['px1', 'py1', 'pz1']
O2_orbs =  ['px2', 'py2', 'pz2']
compute A(w) for symmetries =  ['1A1', '1B1', '3B1', '1A2', '3A2', '1E', '3E']
Mc= 10
VS.dim =  93025
No. of states with count_upup, count_updn, count_dnup, count_dndn: 0 46975 46050 0
len(d_list) 25
len(pUpp_list) 0
len(pUppx_list) 450
len(pUppy_list) 450
len(pVpp_list) 3240
len(pUpdx_list) 60
len(pUpdy_list) 60


Total Progress:   0%|          | 0/132 [00:00<?, ?it/s]

E_1S =  12.16
E_1G =  7.76
E_1D =  6.71
E_3P =  7.05
E_3F =  4.8
A= 6.0 epx= 3.5 epy= 3.5  pds= 1.50111  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0.0  Udif= 0.0  Vpp= 0.0  Upd= 0.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-1.63522934 -0.33761233 -0.33727618 -0.29091604 -0.28918327 -0.28918327
 -0.28918327 -0.28918327 -0.27895355 -0.26183275]
eigenvalue =  -1.6352293410524126
Compute the weights in GS (lowest Aw peak)
no e-h state  15120 px1 -1 0 0 px1 -1 0 0 S= 0 Sz= 0 , weight =  0.010828651222922772
no e-h state  22335 px1 -1 0 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.02772165961831446
no e-h state  22495 py2 0 -1 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.016978272840728087
no e-h state  23438 dx2y2 -2 -1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.0063215374458102744
no e-h state  23468 dx2y2 -2 1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.006321537445810285
no e-h state  23676 dx2y2 -1 -2 0

start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-1.6384335  -0.33889287 -0.33862551 -0.29259481 -0.2906997  -0.2906997
 -0.2906997  -0.2906997  -0.28049598 -0.26259014]
eigenvalue =  -1.6384335048505962
Compute the weights in GS (lowest Aw peak)
no e-h state  15120 px1 -1 0 0 px1 -1 0 0 S= 0 Sz= 0 , weight =  0.010494449434803719
no e-h state  22335 px1 -1 0 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.027648139853321024
no e-h state  22495 py2 0 -1 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.01743250999368459
no e-h state  23438 dx2y2 -2 -1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.0063237464138995525
no e-h state  23468 dx2y2 -2 1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.0063237464138995595
no e-h state  23676 dx2y2 -1 -2 0 px1 0 0 0 S= 0 Sz= 0 , weight =  0.007325380114102545
no e-h state  23706 dx2y2 -1 0 0 px1 0 0 0 S= 0 Sz= 0 , weight =  0.1260884678652895
no e-h state  23736 dx2y2 -1 2 0 px1 0 0 0 S= 0 Sz= 0 , weight 

no e-h state  32943 dx2y2 0 0 0 px1 1 -2 0 S= 0 Sz= 0 , weight =  0.004591418994515211
no e-h state  33844 px1 -1 0 0 px1 1 0 0 S= 0 Sz= 0 , weight =  0.00816596086435228
no e-h state  34008 px1 0 -1 0 py2 1 0 0 S= 0 Sz= 0 , weight =  0.011555993259206365
no e-h state  34026 dx2y2 0 0 0 px1 1 0 0 S= 0 Sz= 0 , weight =  0.13176770264165139
no e-h state  34056 px1 0 1 0 py2 1 0 0 S= 0 Sz= 0 , weight =  0.01155599325920638
no e-h state  34216 px1 1 0 0 px1 1 0 0 S= 0 Sz= 0 , weight =  0.004082980432176152
no e-h state  35127 dx2y2 0 0 0 px1 1 2 0 S= 0 Sz= 0 , weight =  0.0045914189945152
no e-h state  43348 dx2y2 0 0 0 py2 2 -1 0 S= 0 Sz= 0 , weight =  0.0037419251392537835
no e-h state  44593 dx2y2 0 0 0 py2 2 1 0 S= 0 Sz= 0 , weight =  0.003741925139253783
sumweight= 0.972104989696784
E_1S =  12.16
E_1G =  7.76
E_1D =  6.71
E_3P =  7.05
E_3F =  4.8
A= 6.0 epx= 7.01 epy= 6.99  pds= 1.50111  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0.0  Udif= 0.0  Vpp= 0.0  Upd= 0.0
start create_tpd_nn_matrix


...

sumweight= 0.9713727259951024
E_1S =  12.16
E_1G =  7.76
E_1D =  6.71
E_3P =  7.05
E_3F =  4.8
A= 6.0 epx= 7.02 epy= 6.98  pds= 1.50111  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0.0  Udif= 0.0  Vpp= 0.0  Upd= 0.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[1.93278496 2.15846292 2.47477861 2.52674131 2.90798224 3.14136664
 3.41240857 3.43263891 3.4483237  3.7481826 ]
eigenvalue =  1.9327849566585769
Compute the weights in GS (lowest Aw peak)
no e-h state  15120 px1 -1 0 0 px1 -1 0 0 S= 0 Sz= 0 , weight =  0.006731270269163723
no e-h state  22335 px1 -1 0 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.011642572960136698
no e-h state  22495 py2 0 -1 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.004427252822391899
no e-h state  23438 dx2y2 -2 -1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.0045414293230801595
no e-h state  23468 dx2y2 -2 1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.004541429323080156
no e-h stat

sumweight= 0.9718753642680769
E_1S =  12.16
E_1G =  7.76
E_1D =  6.71
E_3P =  7.05
E_3F =  4.8
A= 6.0 epx= 7.05 epy= 6.95  pds= 1.50111  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0.0  Udif= 0.0  Vpp= 0.0  Upd= 0.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[1.93457997 2.15900505 2.4754802  2.52755017 2.90880051 3.14188697
 3.41145476 3.43524264 3.44833615 3.74644379]
eigenvalue =  1.9345799742507137
Compute the weights in GS (lowest Aw peak)
no e-h state  15120 px1 -1 0 0 px1 -1 0 0 S= 0 Sz= 0 , weight =  0.0065732033298268695
no e-h state  22335 px1 -1 0 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.01166491936676616
no e-h state  22495 py2 0 -1 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.004549278102483782
no e-h state  23438 dx2y2 -2 -1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.004560381490978826
no e-h state  23468 dx2y2 -2 1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.004560381490978829
no e-h state

start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-1.63336638 -0.33706933 -0.33668279 -0.29008981 -0.28847344 -0.28847344
 -0.28847344 -0.28847344 -0.27822932 -0.26151677]
eigenvalue =  -1.6333663845350042
Compute the weights in GS (lowest Aw peak)
no e-h state  15120 px1 -1 0 0 px1 -1 0 0 S= 0 Sz= 0 , weight =  0.016679426898862284
no e-h state  22335 px1 -1 0 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.027767158825477575
no e-h state  22495 py2 0 -1 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.011056026376782322
no e-h state  23438 dx2y2 -2 -1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.0074131543565553225
no e-h state  23468 dx2y2 -2 1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.007413154356555327
no e-h state  23676 dx2y2 -1 -2 0 px1 0 0 0 S= 0 Sz= 0 , weight =  0.006318821859446202
no e-h state  23706 dx2y2 -1 0 0 px1 0 0 0 S= 0 Sz= 0 , weight =  0.1536935859964342
no e-h state  23736 dx2y2 -1 2 0 px1 0 0 0 S= 0 Sz= 0 , weight

no e-h state  32943 dx2y2 0 0 0 px1 1 -2 0 S= 0 Sz= 0 , weight =  0.006312878750900987
no e-h state  33844 px1 -1 0 0 px1 1 0 0 S= 0 Sz= 0 , weight =  0.03247440313646248
no e-h state  34008 px1 0 -1 0 py2 1 0 0 S= 0 Sz= 0 , weight =  0.027830061097827524
no e-h state  34026 dx2y2 0 0 0 px1 1 0 0 S= 0 Sz= 0 , weight =  0.15148189124601819
no e-h state  34056 px1 0 1 0 py2 1 0 0 S= 0 Sz= 0 , weight =  0.027830061097827596
no e-h state  34216 px1 1 0 0 px1 1 0 0 S= 0 Sz= 0 , weight =  0.01623720156823126
no e-h state  35127 dx2y2 0 0 0 px1 1 2 0 S= 0 Sz= 0 , weight =  0.006312878750900992
no e-h state  43348 dx2y2 0 0 0 py2 2 -1 0 S= 0 Sz= 0 , weight =  0.007462882193772344
no e-h state  44593 dx2y2 0 0 0 py2 2 1 0 S= 0 Sz= 0 , weight =  0.007462882193772342
sumweight= 0.9270700436030505
E_1S =  12.16
E_1G =  7.76
E_1D =  6.71
E_3P =  7.05
E_3F =  4.8
A= 6.0 epx= 7.0 epy= 7.0  pds= 1.50111  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0.0  Udif= 0.0  Vpp= 0.0  Upd= 0.0
start create_tpd_nn_matrix


no e-h state  32943 dx2y2 0 0 0 px1 1 -2 0 S= 0 Sz= 0 , weight =  0.003745324898859515
no e-h state  33844 px1 -1 0 0 px1 1 0 0 S= 0 Sz= 0 , weight =  0.014155837141154964
no e-h state  34008 px1 0 -1 0 py2 1 0 0 S= 0 Sz= 0 , weight =  0.011575616079617787
no e-h state  34026 dx2y2 0 0 0 px1 1 0 0 S= 0 Sz= 0 , weight =  0.16685760243150252
no e-h state  34056 px1 0 1 0 py2 1 0 0 S= 0 Sz= 0 , weight =  0.011575616079617787
no e-h state  34216 px1 1 0 0 px1 1 0 0 S= 0 Sz= 0 , weight =  0.007077918570577476
no e-h state  35127 dx2y2 0 0 0 px1 1 2 0 S= 0 Sz= 0 , weight =  0.003745324898859514
no e-h state  43348 dx2y2 0 0 0 py2 2 -1 0 S= 0 Sz= 0 , weight =  0.0046067129568222445
no e-h state  44593 dx2y2 0 0 0 py2 2 1 0 S= 0 Sz= 0 , weight =  0.004606712956822233
sumweight= 0.9722967678758846
E_1S =  12.16
E_1G =  7.76
E_1D =  6.71
E_3P =  7.05
E_3F =  4.8
A= 6.0 epx= 7.03 epy= 6.97  pds= 1.50111  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0.0  Udif= 0.0  Vpp= 0.0  Upd= 0.0
start create_tpd_nn_ma